In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.01

In [3]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

In [4]:
X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

In [5]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)
    
    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)

In [6]:
with tf.name_scope("layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [7]:
# cost/loss function
with tf.name_scope("cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

In [8]:
with tf.name_scope("train"):
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [9]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [10]:
# Launch graph
with tf.Session() as sess:
    # 저장된 폴더에 가서
    # tensorboard --logdir=./logs/xor_logs 을 cmd로 입력하여 텐서보드 실행
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph) # Show the graph
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X:x_data,Y:y_data})
        writer.add_summary(summary, global_step=step)
        
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))
            
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.7156377 [array([[ 0.7926959 ,  0.6886104 ],
       [-1.2072834 , -0.29517072]], dtype=float32), array([[1.7177    ],
       [0.35572484]], dtype=float32)]
1000 0.022333965 [array([[ 5.67365  , -6.219753 ],
       [-6.1447105,  6.572153 ]], dtype=float32), array([[8.90876 ],
       [8.472787]], dtype=float32)]
2000 0.006309393 [array([[ 6.6223445, -7.1925664],
       [-7.0854316,  7.5267982]], dtype=float32), array([[11.223158],
       [10.811347]], dtype=float32)]
3000 0.0027709822 [array([[ 7.1489196, -7.7329946],
       [-7.609344 ,  8.0584545]], dtype=float32), array([[12.759907],
       [12.362965]], dtype=float32)]
4000 0.0014247242 [array([[ 7.5337973, -8.127679 ],
       [-7.9929643,  8.447398 ]], dtype=float32), array([[14.016665],
       [13.630341]], dtype=float32)]
5000 0.0007894051 [array([[ 7.849699 , -8.4513235],
       [-8.308202 ,  8.766735 ]], dtype=float32), array([[15.141127],
       [14.763162]], dtype=float32)]
6000 0.00045509645 [array([[ 8.125374, -8.733494],